In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import ImageOps, Image
from google.colab.patches import cv2_imshow
from matplotlib.colors import ListedColormap
from tqdm import tqdm
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
TRAIN_JSON_PATH = '/content/drive/MyDrive/Colab Notebooks/Data/label/train/'
VALID_JSON_PATH = '/content/drive/MyDrive/Colab Notebooks/Data/label/val/'

TRAIN_IMG_PATH = '/content/drive/MyDrive/Colab Notebooks/Data/img/train/'
VALID_IMG_PATH = '/content/drive/MyDrive/Colab Notebooks/Data/img/val/'

TRAIN_SEGMENT_PATH = '/content/drive/MyDrive/Colab Notebooks/myData/segmentation/train/'
VALID_SEGMENT_PATH = '/content/drive/MyDrive/Colab Notebooks/myData/segmentation/val/'

CSV_PATH = '/content/drive/MyDrive/Colab Notebooks/myData/'

PREPROC_TRAIN_IMG_PATH = '/content/drive/MyDrive/Colab Notebooks/myData/preprocessed/train/'

In [ ]:
df = pd.read_csv(CSV_PATH + 'train_ver3.csv')

In [ ]:
happy_npz = np.load(TRAIN_SEGMENT_PATH + f'train_happy.npz')
anger_npz = np.load(TRAIN_SEGMENT_PATH + f'train_anger.npz')
sadness_npz = np.load(TRAIN_SEGMENT_PATH + f'train_sadness.npz')
panic_npz = np.load(TRAIN_SEGMENT_PATH + f'train_panic.npz')

In [ ]:
# train 함수
def segmentation_images(imgfile, emotion):
    file_name = imgfile.split('/')[-1]
    if emotion == 'happy':
      npz = happy_npz
    elif emotion == 'anger':
      npz = anger_npz
    elif emotion == 'sadness':
      npz = sadness_npz
    elif emotion == 'panic':
      npz = panic_npz

    segment_mask = npz[file_name]

    # img = Image.open(imgfile).convert('RGB')
    # img = ImageOps.exif_transpose(img)

    img = cv2.imread(imgfile)
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # np_img = np.array(img)
    img[segment_mask != 3] = 0

    # seg_img = Image.fromarray(np_img)

    return img

In [ ]:
fault_list = []

In [ ]:
def crop_images(filepath, coordinates, emotion): # coordinates = (df['annot_A.boxes.minX'][i], df['annot_A.boxes.minY'][i], df['annot_A.boxes.maxX'][i], df['annot_A.boxes.maxY'][i])
    # img = Image.open(filepath).convert('RGB')
    # img = ImageOps.exif_transpose(img)
    img = cv2.imread(filepath)
    #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    #croped_img = seg_img.crop(coordinates)
    for c in coordinates:
      if c < 0:
        fault_list.append((filepath.split('/')[-1], emotion))
        print(filepath.split('/')[-1])
        del img
        return None
    croped_img = img[int(coordinates[0]):int(coordinates[1]), int(coordinates[2]):int(coordinates[3])]

    del img

    return croped_img

In [ ]:
filenames = list(df['filename'])
emotions = list(df['faceExp_uploader'])

In [ ]:
# # All files segmentation
# for i in tqdm(range(len(filenames))):
#     save_path = PREPROC_TRAIN_IMG_PATH
#     if emotions[i] == 0:
#         emotion = 'happy'
#     elif emotions[i] == 1:
#         emotion = 'anger'
#     elif emotions[i] == 2:
#         emotion = 'sadness'
#     elif emotions[i] == 3:
#         emotion = 'panic'
#     else:
#         print('NOT SUPPORTED EMOTION')
#         continue

#     seg_img = segmentation_images(TRAIN_IMG_PATH + emotion + '/' + filenames[i], emotion)

#     if not os.path.exists(save_path):
#         os.makedirs(save_path)
#     save_path += 'segmentation/'
#     os.makedirs(save_path, exist_ok=True)
#     save_path += emotion + '/'
#     os.makedirs(save_path, exist_ok=True)
#     cv2.imwrite(save_path + filenames[i], seg_img)

In [ ]:
# All files crop
for i in tqdm(range(len(filenames))):
    save_path = PREPROC_TRAIN_IMG_PATH
    emotion = ""
    if emotions[i] == 0:
        emotion += 'happy'
    elif emotions[i] == 1:
        emotion += 'anger'
    elif emotions[i] == 2:
        emotion += 'sadness'
    elif emotions[i] == 3:
        emotion += 'panic'
    else:
        print('NOT SUPPORTED EMOTION')
        continue

    coordinate = (df['minY'][i], df['maxY'][i], df['minX'][i], df['maxX'][i])
    croped_img = crop_images(TRAIN_IMG_PATH + emotion + '/' + filenames[i], coordinate, emotion)
    if croped_img is None:
      continue
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path += 'crop/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    save_path += emotion + '/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    cv2.imwrite(save_path + filenames[i], croped_img)
    del croped_img
    del save_path
    del emotion

100%|██████████| 5992/5992 [2:36:16<00:00,  1.56s/it]


In [ ]:
print('크롭 실패 목록')
print(fault_list)

크롭 실패 목록
[]


In [ ]:
len(fault_list)

0

In [ ]:
# print(PREPROC_TRAIN_IMG_PATH + emotion + '/' + filenames[0])

In [ ]:
# All files segmentation & crop
for i in tqdm(range(len(filenames))):
    save_path = PREPROC_TRAIN_IMG_PATH
    if emotions[i] == 0:
        emotion = 'happy'
    elif emotions[i] == 1:
        emotion = 'anger'
    elif emotions[i] == 2:
        emotion = 'sadness'
    elif emotions[i] == 3:
        emotion = 'panic'
    else:
        print('NOT SUPPORTED EMOTION')
        continue

    coordinate = (df['minY'][i], df['maxY'][i], df['minX'][i], df['maxX'][i])
    croped_img = crop_images(PREPROC_TRAIN_IMG_PATH + 'segmentation/' + emotion + '/' + filenames[i], coordinate, emotion)
    if croped_img is None:
        continue
    mean_val1 = np.mean(croped_img, axis=0)
    mean_val2 = np.mean(mean_val1)
    if mean_val2 < 10:
        print("black image: ", filenames[i], f'index: {i}')
        continue
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path += 'segmentation_and_crop/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    save_path += emotion + '/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    cv2.imwrite(save_path + filenames[i], croped_img)

100%|██████████| 5992/5992 [1:53:30<00:00,  1.14s/it]


In [ ]:
df_val = pd.read_csv(CSV_PATH + 'valid_ver3.csv')

In [ ]:
PREPROC_VALID_IMG_PATH = '/content/drive/MyDrive/Colab Notebooks/myData/preprocessed/val/'

In [ ]:
del happy_npz
del anger_npz
del sadness_npz
del panic_npz

In [ ]:
happy_npz = np.load(VALID_SEGMENT_PATH + f'val_happy.npz')
anger_npz = np.load(VALID_SEGMENT_PATH + f'val_anger.npz')
sadness_npz = np.load(VALID_SEGMENT_PATH + f'val_sadness.npz')
panic_npz = np.load(VALID_SEGMENT_PATH + f'val_panic.npz')

In [ ]:
fault_list.clear()
filenames.clear()
emotions.clear()

In [ ]:
fault_list = []
filenames = list(df_val['filename'])
emotions = list(df_val['faceExp_uploader'])

In [ ]:
# All files segmentation
for i in tqdm(range(len(filenames))):
    save_path = PREPROC_VALID_IMG_PATH
    if emotions[i] == 0:
        emotion = 'happy'
    elif emotions[i] == 1:
        emotion = 'anger'
    elif emotions[i] == 2:
        emotion = 'sadness'
    elif emotions[i] == 3:
        emotion = 'panic'
    else:
        print('NOT SUPPORTED EMOTION')
        continue

    seg_img = segmentation_images(VALID_IMG_PATH + emotion + '/' + filenames[i], emotion)

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path += 'segmentation/'
    os.makedirs(save_path, exist_ok=True)
    save_path += emotion + '/'
    os.makedirs(save_path, exist_ok=True)
    cv2.imwrite(save_path + filenames[i], seg_img)

100%|██████████| 1200/1200 [40:42<00:00,  2.04s/it]


In [ ]:
# All files crop
for i in tqdm(range(len(filenames))):
    save_path = PREPROC_VALID_IMG_PATH
    emotion = ""
    if emotions[i] == 0:
        emotion += 'happy'
    elif emotions[i] == 1:
        emotion += 'anger'
    elif emotions[i] == 2:
        emotion += 'sadness'
    elif emotions[i] == 3:
        emotion += 'panic'
    else:
        print('NOT SUPPORTED EMOTION')
        continue

    coordinate = (df_val['minY'][i], df_val['maxY'][i], df_val['minX'][i], df_val['maxX'][i])
    croped_img = crop_images(VALID_IMG_PATH + emotion + '/' + filenames[i], coordinate, emotion)
    if croped_img is None:
      continue
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path += 'crop/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    save_path += emotion + '/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    cv2.imwrite(save_path + filenames[i], croped_img)
    del croped_img
    del save_path
    del emotion

100%|██████████| 1200/1200 [03:14<00:00,  6.18it/s]


In [ ]:
print('크롭 실패 목록')
print(fault_list)

크롭 실패 목록
[]


In [ ]:
len(fault_list)

0

In [ ]:
# All files segmentation & crop
for i in tqdm(range(len(filenames))):
    save_path = PREPROC_VALID_IMG_PATH
    if emotions[i] == 0:
        emotion = 'happy'
    elif emotions[i] == 1:
        emotion = 'anger'
    elif emotions[i] == 2:
        emotion = 'sadness'
    elif emotions[i] == 3:
        emotion = 'panic'
    else:
        print('NOT SUPPORTED EMOTION')
        continue

    coordinate = (df_val['minY'][i], df_val['maxY'][i], df_val['minX'][i], df_val['maxX'][i])
    croped_img = crop_images(PREPROC_VALID_IMG_PATH + 'segmentation/' + emotion + '/' + filenames[i], coordinate, emotion)
    if croped_img is None:
        continue
    mean_val1 = np.mean(croped_img, axis=0)
    mean_val2 = np.mean(mean_val1)
    if mean_val2 < 10:
        print("black image: ", filenames[i], f'index: {i}')
        continue
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path += 'segmentation_and_crop/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    save_path += emotion + '/'
    if not os.path.exists(save_path):
      os.makedirs(save_path, exist_ok=True)
    cv2.imwrite(save_path + filenames[i], croped_img)

 30%|███       | 362/1200 [00:44<01:19, 10.54it/s]

black image:  rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg index: 359


100%|██████████| 1200/1200 [02:31<00:00,  7.93it/s]


In [ ]:
df_val[df_val['filename']=='rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg']

,Unnamed: 0,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A.faceExp,annot_A.bg,annot_B.faceExp,...,annot_C.faceExp,annot_C.bg,lenX,lenY,minX,maxX,minY,maxY,face.X,face.Y
359,359,rb04c532132d836a981bf48f70b3588730632285f2c97b...,0,50,0,2,6,슬픔,숙박 및 거주공간,슬픔,...,슬픔,숙박 및 거주공간,2640,1980,1689.859096,2543.880496,243.076451,1342.284391,2116.869796,792.680421
